# 03 — Tree-Based Model (XGBoost)

Goal:
- train a strong non-linear model (XGBoost) to capture feature interactions
- evaluate with **Precision–Recall (PR-AUC)**
- save predicted probabilities + `amount` for expected loss analysis (next notebook)

**Why XGBoost here?**
- captures non-linear interactions in latent PCA feature space
- strong for tabular risk scoring
- supports imbalance via `scale_pos_weight`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_auc_score
from xgboost import XGBClassifier

DATA_PATH = '../data/creditcard.csv'
df = pd.read_csv(DATA_PATH)
cols = list(df.columns)
label_col = 'Class' if 'Class' in cols else 'class'
time_col  = 'Time'  if 'Time'  in cols else 'time'
amount_col= 'Amount' if 'Amount' in cols else 'amount'

df[label_col] = pd.to_numeric(df[label_col], errors='coerce').fillna(0).astype(int)

X = df.drop(columns=[label_col])
y = df[label_col].astype(int)
amount_all = df[amount_col].astype(float).values

X_train, X_test, y_train, y_test, amt_train, amt_test = train_test_split(
    X, y, amount_all, test_size=0.2, random_state=42, stratify=y
)
print('Train fraud rate:', y_train.mean())
print('Test  fraud rate:', y_test.mean())

In [ ]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / max(pos, 1)
scale_pos_weight

In [ ]:
# Train/validation split for early stopping
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

xgb = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1,
    reg_alpha=0.0,
    reg_lambda=1.0,
    gamma=0.0,
    objective='binary:logistic',
    eval_metric='aucpr',
    scale_pos_weight=scale_pos_weight,
    tree_method='hist',
    random_state=42
)

xgb.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=50,
    early_stopping_rounds=50
)

proba_test = xgb.predict_proba(X_test)[:, 1]
ap = average_precision_score(y_test, proba_test)
auc = roc_auc_score(y_test, proba_test)
print(f'XGBoost PR-AUC (Average Precision): {ap:.4f}')
print(f'XGBoost ROC-AUC: {auc:.4f}')

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, proba_test)
plt.figure()
plt.plot(recall, precision)
plt.title('Precision–Recall Curve (XGBoost)')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
# Save artifacts for next notebook
artifacts = pd.DataFrame({'y_true': y_test.values, 'proba': proba_test, 'amount': amt_test})
artifacts.to_csv('../data/xgb_test_scoring.csv', index=False)
print('Saved ../data/xgb_test_scoring.csv')

pd.Series(X.columns).to_csv('../data/feature_names.csv', index=False, header=False)
print('Saved ../data/feature_names.csv')

Next: Notebook 04 selects thresholds by **expected loss** and defines an Allow/Flag/Block policy.
